In [ ]:
import pandas as pd
from IPython.display import Image, display

# Crear un DataFrame con una columna que contiene URLs de imágenes
data = {'Nombre': ['Imagen 1', 'Imagen 2'],
        'URL': ['https://images-na.ssl-images-amazon.com/images/S/compressed.photo.goodreads.com/books/1586722975i/2767052.jpg', 'https://ejemplo.com/imagen2.png']}
df = pd.DataFrame(data)

# Función para mostrar imágenes en las celdas del DataFrame
def mostrar_imagen(url):
    display(Image(url=url))

# Aplicar la función 'mostrar_imagen' a la columna 'URL' del DataFrame
df['URL'].apply(mostrar_imagen)
